🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author : Nicolas Grosjean*

*Date : 09/11/2025*

**Description :**

Analyses bus data with C2C activity points :
- Compute distance between them
- Identify C2C activity points not deserved
- Make pretty visualisations to share

In [1]:
# WORKING DIR NEED TO BE SET BEFORE IMPORTING SETTINGS
import os

os.chdir("../..")
print("Working directory set to the root of the project")

Working directory set to the root of the project


In [2]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, box

from src.settings import EPSG_WEB_MERCATOR, EPSG_WGS84

In [3]:
isere_code = "38"

## Read data

In [4]:
from src.processors.osm import OSMBusLinesProcessor, OSMBusStopsProcessor

In [5]:
osm_stops_gdf = OSMBusStopsProcessor.fetch(reload_pipeline=False).set_crs(
    EPSG_WGS84
)  # TODO Remove CRS
osm_stops_gdf.loc[:, "source"] = "OSM"
osm_stops_gdf.head()

,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other,source
0,None,None,135296,Université - IUT-STAPS,None,[],[3922428],M réso,None,POINT (5.77622 45.19738),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
1,None,None,135930,Hôpital Couple Enfant,None,[],"[3333927, 3922430]",M réso,None,POINT (5.74231 45.20065),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
2,None,None,136570,Cap des H',"Arrêt de régulation, non commercial.",[],[],M réso,None,POINT (5.68159 45.21695),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
3,None,None,136597,Place de la Libération,None,[],"[3031947, 3031948, 3044780, 3044781, 3923550, ...",M réso,None,POINT (5.66272 45.20707),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
4,None,None,137073,Centr'Alp 2,None,[],"[8671286, 8671287]",M réso,None,POINT (5.6043 45.3198),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM


In [6]:
osm_lines_gdf = OSMBusLinesProcessor.fetch(reload_pipeline=False)
osm_lines_gdf.head()

,gtfs_id,osm_id,name,from_location,to,network,network_gtfs_id,network_wikidata,operator,colour,text_colour,stop_gtfs_ids,stops_osm_ids,school,geometry,other
0,None,2067887,Ligne A : Gare de Saint-Clair-Les-Roches ⇒ Ron...,Gare de Saint-Clair-Les-Roches,Rond-point Chanas,TPR,None,None,Courriers Rhodaniens / Fayard,e53b1a,None,[],"[1659415935, 8874916309, 11146173165, 11146173...",False,None,"{'charge': None, 'check_date': None, 'comment'..."
1,None,2569190,Ouibus 70 : Grenoble Gare Routière -> Aéroport...,Grenoble - Gare Routière,Aéroport Lyon Saint-Exupéry - Terminal 1,BlaBlaBus,None,Q1653380,Faure Vercors,#ee0064,None,[],"[2617010911, 474827289, 6074566590]",False,None,"{'charge': None, 'check_date': None, 'comment'..."
2,None,2569239,Ouibus 70 : Aéroport Lyon Saint-Exupéry -> Pla...,Aéroport Lyon Saint-Exupéry - Terminal 1,Grenoble - Gare routière,BlaBlaBus,None,Q1653380,Faure Vercors,#ee0064,None,[],"[6074566590, 457759141, 2617010911]",False,None,"{'charge': None, 'check_date': None, 'comment'..."
3,None,2920548,15 : Bois Français => Grenoble (via Chenevières),Saint Ismier - Bois Français,Grenoble - Verdun-Préfecture,M réso,None,Q131689044,VFD,#1f72b9,None,[],"[2299463674, 513946287, 513946283, 513946279, ...",False,None,"{'charge': None, 'check_date': None, 'comment'..."
4,None,2920549,15 : Grenoble => Bois Français (via Chenevières),Grenoble - Verdun-Préfecture,Saint Ismier - Bois Français,M réso,None,Q131689044,VFD,#1f72b9,None,[],"[372746162, 451116247, 1829688368, 1829874475,...",False,None,"{'charge': None, 'check_date': None, 'comment'..."


In [7]:
activity_gdf = gpd.read_parquet("src/data_2/C2C/depart_topos_stops_isere.parquet")
print(f"{len(activity_gdf)} activity points")
activity_gdf.rename(
    columns={"navitia_id": "Id wp", "name": "Name wp", "nombre_de_depart_de_topo": "nbr_topo"},
    inplace=True,
)
activity_gdf = activity_gdf[["Id wp", "Name wp", "geometry", "nbr_topo"]]
activity_gdf.head()

577 activity points


,Id wp,Name wp,geometry,nbr_topo
0,38440,Pierre Blanche,POINT (5.52399 44.88944),3
1,39041,Alpe d'Huez - Falaises du lac Besson,POINT (6.09444 45.11727),5
2,39108,Les Arias d'en bas (Mariande),POINT (6.17616 44.91703),1
3,39178,Cirque inférieur du Boulon,POINT (5.96136 45.19128),1
4,39195,Le Bourg-d'Oisans - Commères,POINT (6.08117 45.0272),11


In [8]:
area_gdf = gpd.read_file("src/data/transportdatagouv/contour-des-departements.geojson")
area_gdf = area_gdf.set_crs(EPSG_WGS84, allow_override=True)
area_gdf.head()

,code,nom,geometry
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
1,02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.7908, 3..."
3,04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
4,05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."


In [9]:
isere_activity_df = (
    gpd.sjoin(
        activity_gdf.to_crs(EPSG_WGS84),
        area_gdf[area_gdf["code"] == isere_code].to_crs(EPSG_WGS84),
    )
    .loc[:, ["Id wp"]]
    .drop_duplicates()
    .merge(activity_gdf, on="Id wp", how="inner")
)
isere_activity_gdf = gpd.GeoDataFrame(isere_activity_df, geometry="geometry").set_crs(
    EPSG_WGS84
)
print(f"Filter on Isère: keep {len(isere_activity_gdf)} on {len(activity_gdf)}")

Filter on Isère: keep 514 on 577


In [10]:
tdg_stops_gdf = gpd.read_parquet(
    os.path.join(os.getcwd(), f"src/data/transportdatagouv/stops_{isere_code}.parquet")
)
tdg_stops_gdf.columns = [
    "network_gtfs_id",
    "network",
    "gtfs_id",
    "name",
    "stop_code",
    "description",
    "line_gtfs_ids",
    "geometry",
]
tdg_stops_gdf.loc[:, "source"] = "TDG"
tdg_stops_gdf.head()

,network_gtfs_id,network,gtfs_id,name,stop_code,description,line_gtfs_ids,geometry,source
0,CARSxREGIONxAIN:Network:1:LOC,REGION - cars Région Ain,FR:38261:ZE:38367:CARSxREGIONxAIN,La gare,1136154,<NA>,{'line_id': 'CARSxREGIONxAIN:FlexibleLine:1005...,POINT (608999.29 5727934.37),TDG
1,CARSxREGIONxAIN:Network:1:LOC,REGION - cars Région Ain,FR:38055:ZE:38540:CARSxREGIONxAIN,Mairie,1137588,<NA>,{'line_id': 'CARSxREGIONxAIN:FlexibleLine:1005...,POINT (615432.365 5731543.476),TDG
2,CARSxREGIONxAIN:Network:1:LOC,REGION - cars Région Ain,FR:38465:ZE:38541:CARSxREGIONxAIN,Place,1137590,<NA>,{'line_id': 'CARSxREGIONxAIN:FlexibleLine:1005...,POINT (612596.334 5731950.939),TDG
3,ARDECHE:Network:1:LOC,REGION - cars Région Ardèche,FR:38298:ZE:40822:ARDECHE,Gare SNCF,1023073,<NA>,"{'line_id': 'ARDECHE:Line:1000601:LOC', 'line_...",POINT (533823.813 5680267.662),TDG
4,ARDECHE:Network:1:LOC,REGION - cars Région Ardèche,FR:38349:ZE:41169:ARDECHE,Mairie,1027195,<NA>,"{'line_id': 'ARDECHE:Line:1000601:LOC', 'line_...",POINT (531434.796 5671771.274),TDG


In [11]:
from src.processors.c2c import C2CBusStopsProcessor

In [12]:
c2c_stops_gdf = C2CBusStopsProcessor.fetch(reload_pipeline=False)
c2c_stops_gdf.loc[:, "source"] = "C2C"
c2c_stops_gdf.head()

,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other,source
0,None,stop_area:OGE:GEN15846,None,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (679047.781 5677868.337),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
1,None,stop_area:OGE:GEN15852,None,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",None,[],[],Mobilités M - TouGo,None,POINT (678910.858 5678497.283),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
2,None,stop_area:OGE:GEN15850,None,"Le Haut-Bréda, Chinfert (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678788.406 5679655.482),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
3,None,stop_area:OGE:GEN15080,None,"Le Haut-Bréda, la Piat (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678378.751 5675153.444),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
4,None,stop_area:OGE:GEN13054,None,"Domène, Domène Mairie (Domène)",None,[],[],Mobilités M - Tag,None,POINT (649900.998 5653440.123),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C


In [13]:
all_stops_gdf = pd.concat(
    (osm_stops_gdf, tdg_stops_gdf.to_crs(EPSG_WGS84), c2c_stops_gdf.to_crs(EPSG_WGS84))
)
all_stops_gdf.tail()

/tmp/ipykernel_6910/3211379376.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stops_gdf = pd.concat(


,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other,source,stop_code
645,None,stop_area:O38:3226468,NaN,École (Auris),None,[],[],Isère - Transisère,None,POINT (6.0865 45.0459),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
646,None,stop_area:O38:3224622,NaN,Les Égaux (Saint-Pierre-de-Chartreuse),None,[],[],Isère - Transisère,None,POINT (5.80145 45.31654),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
647,None,stop_area:O38:3224623,NaN,Gérentière (Saint-Pierre-de-Chartreuse),None,[],[],Isère - Transisère,None,POINT (5.80912 45.31962),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
648,None,stop_area:O38:3224627,NaN,École de Saint-Hugues (Saint-Pierre-de-Chartre...,None,[],[],Isère - Transisère,None,POINT (5.80637 45.32302),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
649,None,stop_area:O38:3224625,NaN,Saint-Hugues (Saint-Pierre-de-Chartreuse),None,[],[],Isère - Transisère,None,POINT (5.80648 45.32368),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>


In [14]:
from src.processors.distances import DistancesProcessor

In [15]:
distance_threshold_km = 5

In [16]:
distances_df = DistancesProcessor.fetch(reload_pipeline=False)
distances_df[distances_df["distance_m"] > distance_threshold_km * 1000] = None
nb_lines_before_drop = len(distances_df)
distances_df = distances_df[~pd.isnull(distances_df["distance_m"])]
print(f"{len(distances_df)} distances kept on {nb_lines_before_drop}")
distances_df.head()

50427 distances kept on 11555748


,osm_id,gtfs_id,navitia_id,Id wp,distance_m
451,135296.0,None,None,1198615.0,3832.8
548,135930.0,None,None,102210.0,2548.2
599,135930.0,None,None,104323.0,2765.7
633,135930.0,None,None,104564.0,4859.1
817,135930.0,None,None,224381.0,3877.0


## Analyse data

In [17]:
activity_gdf["Category"] = "Unreachable"
activity_gdf.loc[~activity_gdf["Id wp"].isin(isere_activity_df["Id wp"]), "Category"] = (
    "Outside departement"
)
activity_gdf.loc[activity_gdf["Id wp"].isin(distances_df["Id wp"].unique()), "Category"] = (
    "Reachable"
)
activity_gdf.groupby("Category").count()["Id wp"]

Category
Outside departement     63
Reachable              479
Unreachable             35
Name: Id wp, dtype: int64

### C2C

In [18]:
near_c2c_stops_gdf = c2c_stops_gdf[
    c2c_stops_gdf["navitia_id"].isin(distances_df["navitia_id"].unique())
]
print(
    f"{len(near_c2c_stops_gdf)} C2C bus stops are near activity points (on {len(c2c_stops_gdf)} bus stops)"
)

642 C2C bus stops are near activity points (on 650 bus stops)


In [19]:
c2c_interesting_columns = ["navitia_id", "name", "network", "geometry", "source", "lines"]
expanded = c2c_stops_gdf["other"].apply(pd.Series)
expanded_c2c_stops_gdf = pd.concat([c2c_stops_gdf.drop(columns=["other"]), expanded], axis=1)
expanded_c2c_stops_gdf["lines"] = expanded_c2c_stops_gdf["stoparea_id_and_line"].apply(
    lambda l: [e["line"] for e in l]
)
expanded_c2c_stops_gdf[c2c_interesting_columns].head()

,navitia_id,name,network,geometry,source,lines
0,stop_area:OGE:GEN15846,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",Mobilités M - TouGo,POINT (679047.781 5677868.337),C2C,[Bus 79 - ALLEVARD ECOLE PLEIADE]
1,stop_area:OGE:GEN15852,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",Mobilités M - TouGo,POINT (678910.858 5678497.283),C2C,[Bus 79 - ALLEVARD ECOLE PLEIADE]
2,stop_area:OGE:GEN15850,"Le Haut-Bréda, Chinfert (Le Haut-Bréda)",Mobilités M - TouGo,POINT (678788.406 5679655.482),C2C,"[Bus 79 - ALLEVARD ECOLE PLEIADE, Bus 79 - ALL..."
3,stop_area:OGE:GEN15080,"Le Haut-Bréda, la Piat (Le Haut-Bréda)",Mobilités M - TouGo,POINT (678378.751 5675153.444),C2C,"[Bus 79 - ALLEVARD ECOLE PLEIADE, Bus 79 - ALL..."
4,stop_area:OGE:GEN13054,"Domène, Domène Mairie (Domène)",Mobilités M - Tag,POINT (649900.998 5653440.123),C2C,[Bus 15 - GRENOBLE Verdun - Préfecture / DOMÈN...


In [20]:
c2c_distances = distances_df.merge(
    expanded_c2c_stops_gdf[c2c_interesting_columns].explode("lines"),
    on="navitia_id",
    how="inner",
)
c2c_distances.head()

,osm_id,gtfs_id,navitia_id,Id wp,distance_m,name,network,geometry,source,lines
0,NaN,None,stop_area:OGE:GEN15846,104317.0,3372.3,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",Mobilités M - TouGo,POINT (679047.781 5677868.337),C2C,Bus 79 - ALLEVARD ECOLE PLEIADE
1,NaN,None,stop_area:OGE:GEN15846,104456.0,3452.4,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",Mobilités M - TouGo,POINT (679047.781 5677868.337),C2C,Bus 79 - ALLEVARD ECOLE PLEIADE
2,NaN,None,stop_area:OGE:GEN15852,104317.0,3855.7,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",Mobilités M - TouGo,POINT (678910.858 5678497.283),C2C,Bus 79 - ALLEVARD ECOLE PLEIADE
3,NaN,None,stop_area:OGE:GEN15852,104456.0,2986.9,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",Mobilités M - TouGo,POINT (678910.858 5678497.283),C2C,Bus 79 - ALLEVARD ECOLE PLEIADE
4,NaN,None,stop_area:OGE:GEN15850,104456.0,3093.8,"Le Haut-Bréda, Chinfert (Le Haut-Bréda)",Mobilités M - TouGo,POINT (678788.406 5679655.482),C2C,Bus 79 - ALLEVARD ECOLE PLEIADE


In [21]:
reachable_c2c_nb = len(c2c_distances[~pd.isnull(c2c_distances["navitia_id"])]["Id wp"].unique())
print(f"{reachable_c2c_nb} activity point access are reachable with C2C data")

411 activity point access are reachable with C2C data


In [22]:
c2c_mask = c2c_distances["distance_m"] == c2c_distances.groupby(["Id wp", "lines"])[
    "distance_m"
].transform("min")
c2c_neareast_stops_by_line_gdf = expanded_c2c_stops_gdf[
    expanded_c2c_stops_gdf["navitia_id"].isin(c2c_distances[c2c_mask]["navitia_id"].unique())
]
print(
    f"{len(c2c_neareast_stops_by_line_gdf)} C2C bus stops are the nearest activity points for their bus line (on {len(c2c_stops_gdf)} bus stops)"
)

423 C2C bus stops are the nearest activity points for their bus line (on 650 bus stops)


### OSM

In [23]:
osm_interesting_columns = [
    "osm_id",
    "name",
    "description",
    "line_osm_ids",
    "network",
    "geometry",
    "source",
]
osm_distances = distances_df.merge(
    osm_stops_gdf[osm_interesting_columns].explode("line_osm_ids"), on="osm_id", how="inner"
)
osm_distances.head()

,osm_id,gtfs_id,navitia_id,Id wp,distance_m,name,description,line_osm_ids,network,geometry,source
0,135296.0,None,None,1198615.0,3832.8,Université - IUT-STAPS,None,3922428,M réso,POINT (5.77622 45.19738),OSM
1,135930.0,None,None,102210.0,2548.2,Hôpital Couple Enfant,None,3333927,M réso,POINT (5.74231 45.20065),OSM
2,135930.0,None,None,102210.0,2548.2,Hôpital Couple Enfant,None,3922430,M réso,POINT (5.74231 45.20065),OSM
3,135930.0,None,None,104323.0,2765.7,Hôpital Couple Enfant,None,3333927,M réso,POINT (5.74231 45.20065),OSM
4,135930.0,None,None,104323.0,2765.7,Hôpital Couple Enfant,None,3922430,M réso,POINT (5.74231 45.20065),OSM


In [24]:
osm_mask = osm_distances["distance_m"] == osm_distances.groupby(["Id wp", "line_osm_ids"])[
    "distance_m"
].transform("min")
osm_neareast_stops_by_line_gdf = osm_stops_gdf[
    osm_stops_gdf["osm_id"].isin(osm_distances[osm_mask]["osm_id"].unique())
]
print(
    f"{len(osm_neareast_stops_by_line_gdf)} OSM bus stops are the nearest activity points for their bus line (on {len(osm_stops_gdf)} bus stops)"
)

1309 OSM bus stops are the nearest activity points for their bus line (on 6806 bus stops)


In [25]:
line_osm_id_to_name = osm_lines_gdf.set_index("osm_id")["name"].to_dict()
osm_neareast_stops_by_line_gdf.loc[:, "lines"] = osm_neareast_stops_by_line_gdf.loc[
    :, "line_osm_ids"
].apply(lambda l: [line_osm_id_to_name[e] for e in l])

/data/hdd/Documents/GitHub/mobilite_durable/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1981: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [26]:
reachable_osm_nb = len(osm_distances[~pd.isnull(osm_distances["osm_id"])]["Id wp"].unique())
print(f"{reachable_osm_nb} activity point access are reachable with OSM data")

435 activity point access are reachable with OSM data


### TDG

In [27]:
tdg_interesting_columns = ["gtfs_id", "name", "line_gtfs_ids", "network", "geometry", "source"]
tdg_distances = distances_df.merge(
    tdg_stops_gdf[tdg_interesting_columns], on="gtfs_id", how="inner"
)
tdg_distances.head()

,osm_id,gtfs_id,navitia_id,Id wp,distance_m,name,line_gtfs_ids,network,geometry,source
0,NaN,XGE,None,43666.0,3762.7,Grenoble - Bus Station,"{'line_id': '4727592809', 'line_name': 'BlaBla...",BlaBlaCar Bus,POINT (636112.298 5651930.353),TDG
1,NaN,XGE,None,102210.0,2687.3,Grenoble - Bus Station,"{'line_id': '4727592809', 'line_name': 'BlaBla...",BlaBlaCar Bus,POINT (636112.298 5651930.353),TDG
2,NaN,XGE,None,102605.0,3646.9,Grenoble - Bus Station,"{'line_id': '4727592809', 'line_name': 'BlaBla...",BlaBlaCar Bus,POINT (636112.298 5651930.353),TDG
3,NaN,XGE,None,104323.0,53.0,Grenoble - Bus Station,"{'line_id': '4727592809', 'line_name': 'BlaBla...",BlaBlaCar Bus,POINT (636112.298 5651930.353),TDG
4,NaN,XGE,None,104424.0,3601.1,Grenoble - Bus Station,"{'line_id': '4727592809', 'line_name': 'BlaBla...",BlaBlaCar Bus,POINT (636112.298 5651930.353),TDG


In [28]:
tdg_mask = tdg_distances["distance_m"] == tdg_distances.groupby(["Id wp", "line_gtfs_ids"])[
    "distance_m"
].transform("min")
tdg_neareast_stops_by_line_gdf = tdg_stops_gdf[
    tdg_stops_gdf["gtfs_id"].isin(tdg_distances[tdg_mask]["gtfs_id"].unique())
]
print(
    f"{len(tdg_neareast_stops_by_line_gdf)} tdg bus stops are the nearest activity points for their bus line (on {len(tdg_stops_gdf)} bus stops)"
)

2228 tdg bus stops are the nearest activity points for their bus line (on 15026 bus stops)


In [29]:
reachable_tdg_nb = len(tdg_distances[~pd.isnull(tdg_distances["gtfs_id"])]["Id wp"].unique())
print(f"{reachable_tdg_nb} activity point access are reachable with TDG data")

471 activity point access are reachable with TDG data


# Visualize

## All activity points

In [30]:
color_mapping = {"Outside departement": "gray", "Reachable": "green", "Unreachable": "purple"}
activity_gdf["color"] = activity_gdf["Category"].apply(lambda c: color_mapping[c])

In [31]:
reachable_nb = sum(activity_gdf["Category"] == "Reachable")
unreachable_nb = sum(activity_gdf["Category"] == "Unreachable")
outside_nb = sum(activity_gdf["Category"] == "Outside departement")
title_html = f"""
    <h4 style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: rgba(255,255,255,0.8); padding: 6px 10px; margin: 0;">
    Points d'accès d'activité CampToCamp.<br>
    - {reachable_nb} points à moins de {distance_threshold_km}km d'un arrêt de bus (<span style=\"color: green\">vert</span>)<br>
    - {unreachable_nb} points à plus de {distance_threshold_km}km d'un arrêt de bus (<span style=\"color: purple\">violet</span>)<br>
    - {outside_nb} points en dehors du département (<span style=\"color: gray\">gris</span>)
    </h4>
"""

In [32]:
# Plot points with bus data far from distance_threshold m

# Add a base OSM map centered on Grenoble
m = folium.Map(
    location=[45.1885, 5.7245], zoom_start=9, tiles="OpenStreetMap", control_scale=True
)

# Add Waymarked Trails hiking layers
folium.TileLayer(tiles="WaymarkedTrails.hiking").add_to(m)

# Add Isère polygon
isere_latlon_coords = [
    (lat, lon)
    for lon, lat in area_gdf[area_gdf["code"] == isere_code]["geometry"]
    .values[0]
    .exterior.coords
]
folium.Polygon(
    locations=isere_latlon_coords,
    color="black",
    weight=3,
    fill_color="black",
    fill_opacity=0.25,
    fill=True,
).add_to(m)

# Add activity markers
columns_to_display = ["Id wp", "Name wp", "nbr_topo"]
folium.GeoJson(
    activity_gdf[columns_to_display + ["color", "geometry"]],
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon="star")),
    tooltip=folium.GeoJsonTooltip(fields=columns_to_display),
    popup=folium.GeoJsonPopup(fields=columns_to_display),
    style_function=lambda x: {
        "markerColor": x["properties"]["color"],
    },
).add_to(m)

# Add title
m.get_root().html.add_child(folium.Element(title_html))

m

In [33]:
m.save("all_activity_points.html")

## Filtered bus data with activities

In [34]:
import json

In [35]:
def list_to_str_with_br(l: list[str], line_number_limit: int = 10) -> str:
    list_to_process = l[:line_number_limit]
    if len(l) > line_number_limit:
        list_to_process += ["..."]
    return "<br>".join(list_to_process)

In [36]:
def get_tdg_line_names(lines: str) -> str:
    lines_json = "[" + lines.replace("'", '"') + "]"
    data = json.loads(lines_json)
    return list_to_str_with_br([d["line_name"] for d in data])

In [37]:
bus_stop_nb = {
    "TDG": len(tdg_neareast_stops_by_line_gdf),
    "OSM": len(osm_neareast_stops_by_line_gdf),
    "C2C": len(c2c_neareast_stops_by_line_gdf),
}
fields_by_source = {
    "TDG": ["gtfs_id", "name", "network", "line_gtfs_ids"],
    "OSM": ["osm_id", "name", "network", "lines"],
    "C2C": ["navitia_id", "name", "network", "lines"],
}
aliases_by_source = {
    "TDG": [f"<b>Arrêt de bus TDG</b><br>ID :", "Nom :", "Réseau :", "Lignes :"],
    "OSM": [f"<b>Arrêt de bus OSM</b><br>ID :", "Nom :", "Réseau :", "Lignes :"],
    "C2C": [f"<b>Arrêt de bus C2C</b><br>ID :", "Nom :", "Réseau :", "Lignes :"],
}
tdg_stops_to_plot = tdg_neareast_stops_by_line_gdf[
    fields_by_source["TDG"] + ["geometry"]
].copy()
tdg_stops_to_plot["line_gtfs_ids"] = tdg_neareast_stops_by_line_gdf["line_gtfs_ids"].apply(
    get_tdg_line_names
)
osm_stops_to_plot = osm_neareast_stops_by_line_gdf[
    fields_by_source["OSM"] + ["geometry"]
].copy()
osm_stops_to_plot["lines"] = osm_stops_to_plot["lines"].apply(list_to_str_with_br)
c2c_stops_to_plot = c2c_neareast_stops_by_line_gdf[
    fields_by_source["C2C"] + ["geometry"]
].copy()
c2c_stops_to_plot["lines"] = c2c_stops_to_plot["lines"].apply(list_to_str_with_br)
gdf_by_source = {"TDG": tdg_stops_to_plot, "OSM": osm_stops_to_plot, "C2C": c2c_stops_to_plot}

In [38]:
title_html = f"""
    <div style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: rgba(255,255,255,0.8); padding: 6px 10px; margin: 0;">
        <h2>Couverture des points d'accès CampToCamp<br>desservis ou non par arrêts de bus<br>(selon sources associés : TDG, OSM, C2C)</h2>
        <h4>
            - {reachable_nb} points à moins de {distance_threshold_km}km d'un arrêt de bus (<span style=\"color: green\">vert</span>)<br>
            - {unreachable_nb} points à plus de {distance_threshold_km}km d'un arrêt de bus (<span style=\"color: purple\">violet</span>)<br>
            <br>
            - {reachable_tdg_nb} points accessibles avec les données TDG<br>
            - {reachable_osm_nb} points accessibles avec les données OSM<br>
            - {reachable_c2c_nb} points accessibles avec les données C2C<br>
        </h4>
    </div>
"""

In [39]:
# Add a base OSM map centered on Grenoble
m = folium.Map(
    location=[45.1885, 5.7245], zoom_start=9, tiles="OpenStreetMap", control_scale=True
)

# Add Waymarked Trails hiking layers
folium.TileLayer(tiles="WaymarkedTrails.hiking", control=False).add_to(m)

# Add Isère polygon
isere_latlon_coords = [
    (lat, lon)
    for lon, lat in area_gdf[area_gdf["code"] == isere_code]["geometry"]
    .values[0]
    .exterior.coords
]
folium.Polygon(
    locations=isere_latlon_coords,
    color="black",
    weight=3,
    fill_color="black",
    fill_opacity=0.25,
    fill=True,
).add_to(m)

# Add bus markers
source_to_id = {"TDG": "gtfs_id", "OSM": "osm_id", "C2C": "navitia_id"}
for source in ["TDG", "OSM", "C2C"]:
    fg = folium.FeatureGroup(name=f"Bus stops from {source} ({bus_stop_nb[source]})", show=True)
    fields = fields_by_source[source]
    aliases = aliases_by_source[source]
    folium.GeoJson(
        gdf_by_source[source][fields + ["geometry"]],
        zoom_on_click=True,
        marker=folium.Marker(icon=folium.Icon(icon="bus", prefix="fa", color="blue")),
        tooltip=folium.GeoJsonTooltip(fields=fields, aliases=aliases),
        popup=folium.GeoJsonPopup(fields=fields, aliases=aliases),
    ).add_to(fg)
    fg.add_to(m)


# Add activity markers
fg = folium.FeatureGroup(name="C2C activity access point", show=True)
aliases = [f"<b>Point d'accès d'activité</b><br>ID :", "Nom :", "Nombre de topos :"]
folium.GeoJson(
    activity_gdf[activity_gdf["Category"] != "Outside departement"][
        columns_to_display + ["color", "geometry"]
    ],
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon="star")),
    tooltip=folium.GeoJsonTooltip(fields=columns_to_display, aliases=aliases),
    popup=folium.GeoJsonPopup(fields=columns_to_display, aliases=aliases),
    style_function=lambda x: {
        "markerColor": x["properties"]["color"],
    },
).add_to(fg)
fg.add_to(m)

# Add layer control to filter layers
folium.LayerControl(collapsed=False).add_to(m)

# Add title
m.get_root().html.add_child(folium.Element(title_html))

m

In [40]:
m.save(f"one_bus_by_line_and_source_at_distance_{distance_threshold_km}_km.html")